In [1]:
import jax
from sklearn.datasets import fetch_openml
import matplotlib
import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import grad
import jax.lax as lax
from jax import random
import struct
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
import h5py

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
x,y = mnist['data'],mnist['target']

x_train,x_test = x[:60000], x[60000:]
y_train,y_test = y[:60000], y[60000:]

x_train =jnp.array(x_train, dtype=jnp.int32)
x_test = jnp.array(x_test,dtype = jnp.int32)
y_train = jnp.array(y_train,dtype = jnp.int32)
y_test = jnp.array(y_test,dtype=jnp.int32)

In [3]:
if jax.devices("gpu"):
    x_train = jax.device_put(x_train, device=jax.devices("gpu")[0])
    y_train = jax.device_put(y_train, device=jax.devices("gpu")[0])
    x_test = jax.device_put(x_test, device=jax.devices("gpu")[0])
    y_test = jax.device_put(y_test, device=jax.devices("gpu")[0])
else:
    print("No GPU available, using CPU.")

In [4]:
x_train = jnp.transpose(jnp.reshape(x_train, (60000, -1)))
x_test = jnp.transpose(jnp.reshape(x_test, (10000, -1)))
y_train = jnp.reshape(y_train, (1, 60000))
y_test = jnp.reshape(y_test, (1, 10000))
x_train = x_train / 255.0
x_test = x_test / 255.0

In [5]:
def one_hot(Y):
  #Y- array of class label containing correct labels
  #computing number of samples in dataset
  Y_size = jnp.size(Y)
  #calculating number of unique class in dataset
  Y_max = jnp.max(Y)
  one_hot_Y = jnp.zeros((Y_size, Y_max + 1), dtype=jnp.int32)
  #sets 0 to 1 at row-jnp.arange(Y_size); column - Y
  one_hot_Y=one_hot_Y.at[jnp.arange(Y_size), Y].set(1)
  one_hot_Y = jnp.transpose(one_hot_Y)
  return one_hot_Y

In [6]:
def initialize_parameters_deep(layer_dims):
   key = jax.random.PRNGKey(0)
   parameters = {}
   L = len(layer_dims)
   for l in range(1, L):
          weight_key, key = jax.random.split(key)
          #parameters['W' + str(l)] =jax.random.normal(key=jax.random.PRNGKey(0), shape=(layer_dims[l], layer_dims[l - 1])) * 0.01
          parameters['W' + str(l)] = jax.random.normal(weight_key, shape=(layer_dims[l], layer_dims[l - 1])) * jnp.sqrt(2/layer_dims[l - 1])
          parameters['b' + str(l)] =jnp.zeros((layer_dims[l],1))
   return parameters

In [7]:
def linear_forward(A, W, b):
    Z=jnp.dot(W,A)+b
    cache=(A,W,b)
    return Z, cache

In [8]:
def relu(z):
  #s=np.maximum(0,z)
  #cache=(z)
  return jnp.maximum(0,z),z

In [9]:
def softmax(z):
   s = jnp.exp(z)/jnp.sum(jnp.exp(z), axis = 0, keepdims = True)
   #activation_cache = (z)
   return s, z

In [10]:
def linear_activation_forward(A_prev, W, b, activation):

     if activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache=relu(Z)
     elif activation == "softmax":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = softmax(Z)
     cache = (linear_cache, activation_cache)

     return A, cache

In [23]:
def L_model_forward(X, parameters):
   caches = []
   A = X
   L = len(parameters) //2
   for l in range(1,L):
     A_prev=A
     A, cache= linear_activation_forward(A_prev, parameters["W"+str(l)],parameters["b"+str(l)],"relu")
     caches.append(cache)
   AL, cache= linear_activation_forward(A, parameters["W"+str(L)],parameters["b"+str(L)],"softmax")

   caches.append(cache)

   return AL,caches

In [12]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -jnp.sum(Y * jnp.log(AL) + (1 - Y) * jnp.log(1 - AL)) / m
    #jnp.squeeze(cost)

    return cost

In [13]:
def linear_backward(dZ, cache):
   A_prev, W, b = cache
   m = A_prev.shape[1]
   dW=(1/m)*(jnp.dot(dZ,A_prev.T))
   db=(1/m)*jnp.sum(dZ,axis=1,keepdims=True)
   dA_prev=jnp.dot(W.T,dZ)
   return dA_prev,dW,db

In [14]:
def softmax_backward(AL, Y):
    dZ = AL- Y
    return dZ

In [25]:
def relu_backward(dA, cache):
    Z = cache
    dZ = jnp.where(Z > 0, dA, 0)
    return dZ

In [16]:
def L_model_backward(AL, Y, caches):
  Y = one_hot(Y)
  grads = {}
  L = len(caches)
  m = AL.shape[1]
  Y = Y.reshape(AL.shape)
  #dAL = -Y/AL
  #dAL=-(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
  current_cache = caches[L-1]
  linear_cache, activation_cache =current_cache
  dZ=softmax_backward(AL,Y)
  dA_prev_temp, dW_temp, db_temp =linear_backward(dZ,linear_cache)
  grads["dA" + str(L-1)] = dA_prev_temp
  grads["dW" + str(L)] = dW_temp
  grads["db" + str(L)] = db_temp
  for l in reversed(range(L-1)):
    current_cache = caches[l]
    linear_cache, activation_cache =current_cache
    dZ=relu_backward( dA_prev_temp,activation_cache)
    dA_prev_temp, dW_temp, db_temp =linear_backward(dZ,linear_cache)
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l+1)] = dW_temp
    grads["db" + str(l+1)] = db_temp
  return grads

In [22]:
def update_parameters(params, grads, learning_rate):
  parameters = params.copy()
  L = len(parameters) // 2
  for l in range(L):
        parameters["W" + str(l+1)] =parameters["W" + str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] =parameters["b" + str(l+1)]-learning_rate*grads["db"+str(l+1)]

        # YOUR CODE ENDS HERE
  return parameters

In [18]:
def get_predictions(Y_hat):
    return jnp.argmax(Y_hat,0)

def get_accuracy(predictions, Y):
    predictions = predictions.reshape(1, predictions.shape[0])
    check = jnp.equal(Y[predictions, jnp.arange(Y.shape[1])], 1)
    accuracy = jnp.mean(check) * 100
    return accuracy


In [19]:
layers_dims = [784,40,20, 10]

In [20]:

def L_layer_model(X, Y, layers_dims, learning_rate = 0.75, num_iterations = 3000,print_cost=False):
  #np.random.seed(1)
  grads = {}
  costs = []
  m = X.shape[1]
  #print(layers_dims)
 # cost = 2.5
  parameters=initialize_parameters_deep(layers_dims)
  for i in range(0, num_iterations):
    AL, caches=L_model_forward(X, parameters)
    grads=L_model_backward(AL, Y, caches)
    parameters=update_parameters(parameters, grads, learning_rate)
    cost = compute_cost(AL,Y)
#    Y_predict = jnp.zeros(AL.shape)
    Y_predict = jnp.zeros_like(AL)
    indices = (jnp.argmax(AL, axis=0), jnp.arange(AL.shape[1]))
    Y_predict = Y_predict.at[indices].set(1)
   # Y_predict = jax.ops.index_update(Y_predict, indices, 1)
   # Y_predict[jnp.argmax(AL, axis = 0), jnp.arange(AL.shape[1])] = 1
   # if i%100:
    #  learning_rate=learning_rate/(1+0.0002)
    if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, jnp.squeeze(cost)))
           # print(learning_rate)
            #print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_predict - Y)) * 100))
            print("accuracy : " , get_accuracy(get_predictions(AL),Y))

    if i % 100 == 0 or i == num_iterations:
            costs.append(cost)

  return parameters,costs, Y_predict,AL

In [26]:
parameters, costs, Y_predict,AL =L_layer_model(x_train,y_train, layers_dims, 0.2, 6000, True)
#print("accuracy : " , get_accuracy(get_predictions(AL),Y_train))

Cost after iteration 0: 103.20845031738281
accuracy :  11.236667
Cost after iteration 100: 288.9808044433594
accuracy :  11.236667
Cost after iteration 200: 323.9647216796875
accuracy :  11.236667
Cost after iteration 300: 343.4630126953125
accuracy :  11.236667
Cost after iteration 400: 358.5415344238281
accuracy :  11.236667
Cost after iteration 500: inf
accuracy :  11.236667
Cost after iteration 600: nan
accuracy :  11.236667
Cost after iteration 700: inf
accuracy :  11.236667
Cost after iteration 800: nan
accuracy :  11.236667
Cost after iteration 900: inf
accuracy :  11.236667
Cost after iteration 1000: nan
accuracy :  11.236667
Cost after iteration 1100: nan
accuracy :  11.236667
Cost after iteration 1200: nan
accuracy :  11.236667
Cost after iteration 1300: nan
accuracy :  11.236667
Cost after iteration 1400: nan
accuracy :  11.236667
Cost after iteration 1500: nan
accuracy :  11.236667
Cost after iteration 1600: nan
accuracy :  11.236667
Cost after iteration 1700: nan
accuracy 